# Monitoring loss of tropical forest cover from Sentinel-1 time-series: A CuSum-based approach

* **Products used:** 
[s1_rtc](https://explorer.digitalearth.africa/products/s1_rtc)

## Background

The Cumulative Sum (CuSum) algorithm is a change point detection method based on time-series analysis. The **Cumulative Sum** change detection algorithm analyses the temporal stability of the signal through the deviation of a variable to its mean.

The CuSum method allows the detection of any type of variation (slow, abrupt) as long as it has an impact on the trend of the time-series. This method has been found to be less affected by the seasonal variability of vegetation and thus more performant to detect abrupt changes in the vegetation structure due to forest cut  [(Ruiz-Ramos et al., 2020)](https://doi.org/10.3390/RS12183061).


## Description
A _compulsory_ description of the notebook, including a brief overview of how Digital Earth Africa helps to address the problem set out above.
It can be good to include a run-down of the tools/methods that will be demonstrated in the notebook:

1. First we do this
2. Then we do this
3. Finally we do this

***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages

Import Python packages that are used for the analysis.

In [1]:
%matplotlib inline

import datacube
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import rioxarray
import math
from shapely.geometry import box
from datacube.utils.geometry import CRS, Geometry
from rioxarray.merge import merge_arrays
from deafrica_tools.datahandling import load_ard
from deafrica_tools.plotting import display_map
from deafrica_tools.dask import create_local_dask_cluster

## Set up a Dask cluster

Dask can be used to better manage memory use and conduct the analysis in parallel. 
For an introduction to using Dask with Digital Earth Africa, see the [Dask notebook](../Beginners_guide/08_Parallel_processing_with_dask.ipynb).

>**Note**: We recommend opening the Dask processing window to view the different computations that are being executed; to do this, see the *Dask dashboard in DE Africa* section of the [Dask notebook](../Beginners_guide/08_Parallel_processing_with_dask.ipynb).

To use Dask, set up the local computing cluster using the cell below.

In [2]:
create_local_dask_cluster()

Client Scheduler: tcp://127.0.0.1:32943 Dashboard: /user/victoria@kartoza.com/proxy/8787/status,Cluster Workers: 1 Cores: 15 Memory: 104.37 GB


### Analysis parameters

An *optional* section to inform the user of any parameters they'll need to configure to run the notebook:

* `param_name_1`: Simple description (e.g. `example_value`). Advice about appropriate values to choose for this parameter.
* `param_name_2`: Simple description (e.g. `example_value`). Advice about appropriate values to choose for this parameter.


In [3]:
# Default study area is the Industrie Foresti`ere du Congo (IFCO) COD 018/11 forest concession (Alibuku)

# Define the central lattitude and longitude of the study area.
central_lat = 0.92045
central_lon = 25.43895

# Define the number of degrees to load around the central latitude and longitude.
lat_buffer = 0.01715
lon_buffer = 0.02674999999999983

# Define the study area.
lat_range = (central_lat - lat_buffer, central_lat + lat_buffer)
lon_range = (central_lon - lon_buffer, central_lon + lon_buffer)

measurements = ["vh", "vv"]
orbit_direction = "descending"
time_range = ("01-01-2018", "01-01-2020")
output_crs = "EPSG:6933"
resolution = (-10, 10)
dask_chunks = dict(x=1000, y=1000)

In [4]:
# View the study area
display_map(x=lon_range, y=lat_range)

### Connect to the datacube

Connect to the datacube so we can access DE Africa data.
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [5]:
dc = datacube.Datacube(app="Forest Monitoring")

## Load the Global Forest Change data

In [6]:
# Function to load Global Forest Change data.
def load_global_forest_change(layer='lossyear', lat_range=None, lon_range=None, geom=None, dask_chunks=None):

    # Check if the layer requested is a valid layer. 
    layers = ['treecover2000', 'gain', 'lossyear', 'datamask', 'first', 'last']
    if layer not in layers: 
        raise ValueError(f"Invalid layer chosen. Please choose a layer from the following layers: {layers}")
    else:
        base_url = f"https://storage.googleapis.com/earthenginepartners-hansen/GFC-2021-v1.9/Hansen_GFC-2021-v1.9_{layer}_"

    # Get the coordinates of the top-left corner for each Global Forest Change tile,
    # covering the area of interest. 
    if geom is None:
        min_lat, max_lat = lat_range[0], lat_range[1]
        min_lon, max_lon = lon_range[0], lon_range[1]
    else: 
        min_lat, max_lat = geom.boundingbox.bottom, geom.boundingbox.top
        min_lon, max_lon = geom.boundingbox.left, geom.boundingbox.right

    lats = np.arange(np.floor(min_lat / 10) * 10, np.ceil(max_lat / 10) * 10, 10).astype(int)
    lons = np.arange(np.floor(min_lon / 10) * 10, np.ceil(max_lon / 10) * 10, 10).astype(int)

    coord_list = []
    for lat in lats:
        lat = lat + 10
        if lat >= 0:
            lat_str = f"{lat:02d}N"
        else:
            lat_str = f"{abs(lat):02d}S"
        for lon in lons:
            if lon >= 0: 
                lon_str = f"{lon:03d}E"
            else:
                lon_str = f"{abs(lon):03d}W"
            coord_str = f"{lat_str}_{lon_str}"
            coord_list.append(coord_str)

    # Load each tile.
    tile_list = []
    for coord in coord_list:
        tile_url = f"{base_url}{coord}.tif"
        # Load the tile as an xarray.DataArray.
        if dask_chunks is not None:
            tile = rioxarray.open_rasterio(tile_url, chunks=dask_chunks).squeeze()
        else:
            tile = rioxarray.open_rasterio(tile_url).squeeze()
        tile_list.append(tile)

    # Merge the tiles into a single xarray.DataArray.
    ds = xr.combine_by_coords(tile_list)
    # Clip the dataset using the bounds of the area of interest.
    ds = ds.rio.clip_box(minx=min_lon, miny=min_lat, maxx=max_lon+0.00025, maxy=max_lat+0.00025)
    # Rename the y and x variables for DEA convention on xarray.DataArrays where crs="EPSG:4326"
    ds = ds.rename({"y":"latitude", "x":"longitude"})
    return ds 

In [7]:
ds_gfc = load_global_forest_change(layer='lossyear', lat_range=lat_range, lon_range=lon_range)
# Reproject the dataset to the output crs.
ds_gfc = ds_gfc.rio.reproject(output_crs)
ds_gfc

<xarray.DataArray (y: 166, x: 196)>
array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ..., 16, 16, 17],
       [ 0,  0, 20, ..., 16, 16, 16],
       [ 0,  0, 20, ..., 16, 16, 16]], dtype=uint8)
Coordinates:
  * x            (x) float64 2.452e+06 2.452e+06 ... 2.457e+06 2.457e+06
  * y            (y) float64 1.196e+05 1.196e+05 ... 1.153e+05 1.153e+05
    band         int64 1
    spatial_ref  int64 0
Attributes:
    LAYER_TYPE:    athematic
    scale_factor:  1.0
    add_offset:    0.0
    long_name:     Layer_1
    _FillValue:    255

In [8]:
# Create a datacube query using the analysis parameters.
query = {
    "measurements": measurements,
    "sat_orbit_state": orbit_direction,
    "time": time_range,
    "like": ds_gfc.geobox,
    #"y": lat_range,
    #"x": lon_range,
    #"output_crs": output_crs,
    #"resolution": resolution,
    #"dask_chunks": dask_chunks,
}

In [9]:
ds_s1 = load_ard(dc=dc, products=["s1_rtc"], group_by="solar_day", **query)

ds_s1

Using pixel quality parameters for Sentinel 1
Finding datasets
    s1_rtc
Applying pixel quality/cloud mask
Loading 60 time steps


<xarray.Dataset>
Dimensions:      (time: 60, y: 166, x: 196)
Coordinates:
  * time         (time) datetime64[ns] 2018-01-05T04:00:54.918330 ... 2019-12...
  * y            (y) float64 1.196e+05 1.196e+05 ... 1.153e+05 1.153e+05
  * x            (x) float64 2.452e+06 2.452e+06 ... 2.457e+06 2.457e+06
    spatial_ref  int32 6933
Data variables:
    vh           (time, y, x) float32 0.07789 0.1043 0.07675 ... 0.04776 0.05634
    vv           (time, y, x) float32 0.1664 0.2913 0.2556 ... 0.1891 0.2129
Attributes:
    crs:           PROJCS["WGS 84 / NSIDC EASE-Grid 2.0 Global",GEOGCS["WGS 8...
    grid_mapping:  spatial_ref

## Change detection algorithm

### CuSum algorithm

In [10]:
def cumsum_algorithm(time_series):
    """
    Takes a numpy array time series and applies the Cumulative
    Sum algorithm described in B. Ygorra et al. 2021.

    Last Modified: July 2022

    Parameters
    ----------
    time_series : numpy array
                  A 3 dimensional numpy array.


    Returns
    -------
    amplitude : numpy array
                A 2 dimensional numpy array.

    """
    # Get the mean of the time series over each pixel.
    time_series_mean = np.mean(time_series, axis=0)

    # Get the time series residuals.
    residuals = time_series - time_series_mean

    # Cumulative sum of the residuals.
    cumsum_residuals = np.cumsum(residuals, axis=0)

    # Determine the maximum and minimum value of the cumulative sum of the residuals.
    max_cumsum_residuals = np.max(cumsum_residuals, axis=0)
    min_cumsum_residuals = np.min(cumsum_residuals, axis=0)

    # Compute the amplitude of the time series.
    amplitude = max_cumsum_residuals - min_cumsum_residuals

    return amplitude


def cumsum_with_bootsrap(ds, band, critical_threshold):
    """
    Takes an xarray.Dataset time series and applies the Cumulative
    Sum algorithm and bootstrapping analysis described in B. Ygorra et al. 2021.

    Last Modified: July 2022

    Parameters
    ----------
    ds : xarray Dataset
         A multi-dimensional array.

    band : str
           Spectral band on which to apply the cumsum algorithm
    
    critical_threshold : float
                         Threshold over which change is considered valid.

    Returns
    -------
    amplitude : numpy array

    """
     # Check if the xarray.DataArray is a valid time series.
    dimensions = ds[band].sizes

    if "time" not in dimensions or dimensions["time"] == 1:
        raise Exception('Please pass a valid time series to the "ds" parameter.')

    # Convert the xarray.DataArray to a numpy array.
    time_series = ds[band].data

    # Get the number of images in the time series.
    number_of_images = dimensions["time"]

    # Compute the original amplitude of the time series.
    original_amplitude = cumsum_algorithm(time_series)

    # The bootstrap consists in conducting CuSum on a randomly modified backscatter timeseries n_bootstraps times.
    # Get the number of boostraps.
    if math.factorial(number_of_images) < 1500:
        n_bootstraps = math.factorial(number_of_images)
    else:
        n_bootstraps = 1500

    # Numpy array to store the count for amplitude_difference > 0. 
    amplitude_count = np.zeros(shape=(n_bootstraps,time_series.shape[1], time_series.shape[2]))

    for n in range(n_bootstraps):
        print(f"Bootstrap Analysis {n+1:02d}/{n_bootstraps}",  end="\r")
        # Randomly organize the the original backscatter time-series thus modifying the temporal order.
        # np.random.shuffle shuffles the numpy array along the first axis (our time axis) of a multi-dimensional array.
        np.random.shuffle(time_series)
        # Apply the CuSum method to the newly reorganized time series.
        amplitude = cumsum_algorithm(time_series)
        # Compute the difference in amplitude between the original time series and the reorganized time series.
        amplitude_difference = original_amplitude - amplitude

        # If amplitude_difference > 0 then original_amplitude > amplitude.
        # This means that original_amplitude is affected by the temporal dimension.
        # Count the number of times amplitude_difference > 0. 
        amplitude_count[n] = np.where(amplitude_difference > 0, 1, 0)

    # The number of times amplitude_difference > 0 is estimated and referred to as the index n_gj.
    # It is an indirect measure of the sequence effect in the backscatter time-series and a
    # sensitivity parameter that intervenes in the computation of the Confidence Level.
    n_gj = amplitude_count.sum(axis=0)
    # The Confidence Level represents the ratio of bootstraps in which the original backscatter
    # time-series presents the original_ampltitude > amplitude in comparison to the total number of bootstraps.
    cl = n_gj / n_bootstraps
    # Convert the Confidence Level numpy array to an xarray.DataArray.
    ds_cl = xr.DataArray(data=cl,
                         dims=["y", "x"],
                         coords=dict(y=("y", ds.y.values),
                                     x=("x", ds.x.values),
                                    spatial_ref=ds.spatial_ref.values),
                         attrs=ds.attrs)
    # A critical threshold value (Tc) can be set as a Confidence Level over which the change point is considered as
    # valid by the bootstrap analysis.
    ds_cl = xr.where(ds_cl>critical_threshold, 1, 0)

    return ds_cl

In [50]:
%%time
cl_vh = cumsum_with_bootsrap(ds=ds_s1, band="vh", critical_threshold=0.5)
cl_vv = cumsum_with_bootsrap(ds=ds_s1, band="vv", critical_threshold=0.5)

CPU times: user 5min 20s, sys: 1min 3s, total: 6min 24s
Wall time: 6min 13s


## Spatial operations over the cumsum results

In [47]:
vv_vh_intersect = xr.where((cl_vh==1) & (cl_vv==1), 1, np.nan)
vv_vh_union = xr.where((cl_vh==1) | (cl_vv==1), 1, np.nan)

## Validate the change results

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

**Last Tested:**

In [ ]:
from datetime import datetime

datetime.today().strftime("%Y-%m-%d")